# Database Statistics and Analytics

This notebook provides insights into your PGVector RAG database, including statistics, visualizations, and performance metrics.

## Setup

In [2]:
import os
import sys
import json
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add parent directory to path
sys.path.append('..')
from pgvector_rag import PGVectorRAG

# Load environment variables
load_dotenv('../.env')

# Set up plotting
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

ModuleNotFoundError: No module named 'plotly'

In [ ]:
# Initialize database connection
conn_params = {
    "host": os.getenv('DB_HOST', 'postgres-pgvector.pgvector.svc.cluster.local'),
    "port": int(os.getenv('DB_PORT', '5432')),
    "database": os.getenv('DB_NAME', 'vectordb'),
    "user": os.getenv('DB_USER', 'vectoruser'),
    "password": os.getenv('DB_PASSWORD', 'vectorpass')
}

# Direct database connection for custom queries
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

# RAG client
rag = PGVectorRAG(conn_params)

print("Connected to database")

## Overall Database Statistics

In [ ]:
# Get overall statistics
cur.execute("""
    SELECT 
        COUNT(DISTINCT project_id) as total_projects,
        COUNT(DISTINCT document_id) as total_documents,
        COUNT(*) as total_chunks,
        AVG(LENGTH(chunk_text)) as avg_chunk_length,
        MAX(LENGTH(chunk_text)) as max_chunk_length,
        MIN(LENGTH(chunk_text)) as min_chunk_length,
        COUNT(DISTINCT topic) as total_topics,
        pg_size_pretty(pg_total_relation_size('document_chunks')) as table_size
    FROM document_chunks
""")

stats = cur.fetchone()
columns = ['total_projects', 'total_documents', 'total_chunks', 'avg_chunk_length', 
           'max_chunk_length', 'min_chunk_length', 'total_topics', 'table_size']

stats_dict = dict(zip(columns, stats))

print("📊 DATABASE OVERVIEW")
print("=" * 50)
for key, value in stats_dict.items():
    if isinstance(value, float):
        print(f"{key.replace('_', ' ').title()}: {value:,.0f}")
    else:
        print(f"{key.replace('_', ' ').title()}: {value}")

## Project-Level Statistics

In [ ]:
# Get statistics by project
cur.execute("""
    SELECT 
        p.id as project_id,
        p.name as project_name,
        COUNT(DISTINCT dc.document_id) as num_documents,
        COUNT(dc.id) as num_chunks,
        AVG(LENGTH(dc.chunk_text)) as avg_chunk_length,
        COUNT(DISTINCT dc.topic) as num_topics,
        MIN(dc.created_at) as first_chunk,
        MAX(dc.created_at) as last_chunk
    FROM projects p
    LEFT JOIN document_chunks dc ON p.id = dc.project_id
    GROUP BY p.id, p.name
    ORDER BY num_chunks DESC
""")

project_stats = pd.DataFrame(
    cur.fetchall(),
    columns=['project_id', 'project_name', 'num_documents', 'num_chunks', 
             'avg_chunk_length', 'num_topics', 'first_chunk', 'last_chunk']
)

print("📁 PROJECT STATISTICS")
print(project_stats.to_string(index=False))

In [ ]:
# Visualize project sizes
if not project_stats.empty:
    fig = px.bar(project_stats, 
                 x='project_name', 
                 y='num_chunks',
                 title='Chunks per Project',
                 labels={'num_chunks': 'Number of Chunks', 'project_name': 'Project'},
                 color='num_documents',
                 color_continuous_scale='Blues')
    fig.show()

## Document Analysis

In [ ]:
# Get document statistics
cur.execute("""
    SELECT 
        document_name,
        COUNT(*) as chunk_count,
        AVG(LENGTH(chunk_text)) as avg_chunk_size,
        MIN(page_number) as first_page,
        MAX(page_number) as last_page,
        COUNT(DISTINCT topic) as topics_covered
    FROM document_chunks
    GROUP BY document_name
    ORDER BY chunk_count DESC
    LIMIT 20
""")

doc_stats = pd.DataFrame(
    cur.fetchall(),
    columns=['document_name', 'chunk_count', 'avg_chunk_size', 
             'first_page', 'last_page', 'topics_covered']
)

print("📄 TOP 20 DOCUMENTS BY CHUNK COUNT")
print(doc_stats.head(10).to_string(index=False))

In [ ]:
# Chunk size distribution
cur.execute("""
    SELECT LENGTH(chunk_text) as chunk_length
    FROM document_chunks
    WHERE LENGTH(chunk_text) > 0
""")

chunk_lengths = [row[0] for row in cur.fetchall()]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(chunk_lengths, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Chunk Length (characters)')
plt.ylabel('Frequency')
plt.title('Distribution of Chunk Lengths')
plt.axvline(np.mean(chunk_lengths), color='red', linestyle='--', label=f'Mean: {np.mean(chunk_lengths):.0f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(chunk_lengths)
plt.ylabel('Chunk Length (characters)')
plt.title('Chunk Length Box Plot')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nChunk Length Statistics:")
print(f"Mean: {np.mean(chunk_lengths):.0f} characters")
print(f"Median: {np.median(chunk_lengths):.0f} characters")
print(f"Std Dev: {np.std(chunk_lengths):.0f} characters")

## Topic Analysis

In [ ]:
# Get topic distribution
cur.execute("""
    SELECT 
        COALESCE(topic, 'Uncategorized') as topic,
        COUNT(*) as chunk_count,
        COUNT(DISTINCT document_id) as document_count,
        AVG(LENGTH(chunk_text)) as avg_chunk_length
    FROM document_chunks
    GROUP BY topic
    ORDER BY chunk_count DESC
""")

topic_stats = pd.DataFrame(
    cur.fetchall(),
    columns=['topic', 'chunk_count', 'document_count', 'avg_chunk_length']
)

print("🏷️  TOPIC DISTRIBUTION")
print(topic_stats.to_string(index=False))

In [ ]:
# Visualize topic distribution
if not topic_stats.empty:
    # Pie chart for chunk distribution
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Chunks by Topic', 'Documents by Topic'),
        specs=[[{'type':'pie'}, {'type':'pie'}]]
    )
    
    fig.add_trace(
        go.Pie(labels=topic_stats['topic'], values=topic_stats['chunk_count'], name='Chunks'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Pie(labels=topic_stats['topic'], values=topic_stats['document_count'], name='Documents'),
        row=1, col=2
    )
    
    fig.update_layout(height=400, title_text="Topic Distribution")
    fig.show()

## Temporal Analysis

In [ ]:
# Get ingestion timeline
cur.execute("""
    SELECT 
        DATE(created_at) as date,
        COUNT(*) as chunks_added,
        COUNT(DISTINCT document_id) as documents_added
    FROM document_chunks
    WHERE created_at IS NOT NULL
    GROUP BY DATE(created_at)
    ORDER BY date
""")

timeline = pd.DataFrame(
    cur.fetchall(),
    columns=['date', 'chunks_added', 'documents_added']
)

if not timeline.empty:
    timeline['date'] = pd.to_datetime(timeline['date'])
    
    # Plot ingestion timeline
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=timeline['date'],
        y=timeline['chunks_added'],
        name='Chunks',
        mode='lines+markers'
    ))
    
    fig.add_trace(go.Scatter(
        x=timeline['date'],
        y=timeline['documents_added'],
        name='Documents',
        mode='lines+markers',
        yaxis='y2'
    ))
    
    fig.update_layout(
        title='Document Ingestion Timeline',
        xaxis_title='Date',
        yaxis_title='Chunks Added',
        yaxis2=dict(
            title='Documents Added',
            overlaying='y',
            side='right'
        ),
        hovermode='x unified'
    )
    
    fig.show()

In [ ]:
# Check for documents with TTL
cur.execute("""
    SELECT 
        COUNT(*) as total_with_ttl,
        MIN(expires_at) as earliest_expiration,
        MAX(expires_at) as latest_expiration,
        COUNT(CASE WHEN expires_at < CURRENT_TIMESTAMP THEN 1 END) as already_expired,
        COUNT(CASE WHEN expires_at BETWEEN CURRENT_TIMESTAMP AND CURRENT_TIMESTAMP + INTERVAL '30 days' THEN 1 END) as expiring_soon
    FROM document_chunks
    WHERE expires_at IS NOT NULL
""")

ttl_stats = cur.fetchone()

if ttl_stats[0] > 0:  # If there are documents with TTL
    print("\n⏰ TTL STATISTICS")
    print("=" * 50)
    print(f"Total chunks with TTL: {ttl_stats[0]}")
    print(f"Already expired: {ttl_stats[3]}")
    print(f"Expiring in next 30 days: {ttl_stats[4]}")
    print(f"Earliest expiration: {ttl_stats[1]}")
    print(f"Latest expiration: {ttl_stats[2]}")

## Vector Index Statistics

In [ ]:
# Get index information
cur.execute("""
    SELECT 
        indexname,
        pg_size_pretty(pg_relation_size(indexname::regclass)) as index_size,
        idx_scan as times_used,
        idx_tup_read as tuples_read,
        idx_tup_fetch as tuples_fetched
    FROM pg_stat_user_indexes
    WHERE schemaname = 'public' AND tablename = 'document_chunks'
    ORDER BY idx_scan DESC
""")

index_stats = pd.DataFrame(
    cur.fetchall(),
    columns=['index_name', 'index_size', 'times_used', 'tuples_read', 'tuples_fetched']
)

print("🔍 INDEX USAGE STATISTICS")
print(index_stats.to_string(index=False))

In [ ]:
# Check vector completeness
cur.execute("""
    SELECT 
        COUNT(*) as total_chunks,
        COUNT(dense_embedding) as chunks_with_dense,
        COUNT(sparse_embedding) as chunks_with_sparse,
        COUNT(CASE WHEN dense_embedding IS NOT NULL AND sparse_embedding IS NOT NULL THEN 1 END) as chunks_with_both,
        COUNT(CASE WHEN dense_embedding IS NULL AND sparse_embedding IS NULL THEN 1 END) as chunks_with_neither
    FROM document_chunks
""")

vector_stats = cur.fetchone()
vector_labels = ['Total Chunks', 'With Dense Embeddings', 'With Sparse Embeddings', 
                 'With Both', 'With Neither']

# Create bar chart
plt.figure(figsize=(10, 6))
bars = plt.bar(vector_labels, vector_stats)
plt.ylabel('Number of Chunks')
plt.title('Vector Embedding Coverage')
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for bar, value in zip(bars, vector_stats):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(vector_stats), 
             f'{value:,}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Calculate percentages
total = vector_stats[0]
if total > 0:
    print("\n📊 EMBEDDING COVERAGE")
    print("=" * 50)
    print(f"Dense embedding coverage: {vector_stats[1]/total*100:.1f}%")
    print(f"Sparse embedding coverage: {vector_stats[2]/total*100:.1f}%")
    print(f"Both embeddings: {vector_stats[3]/total*100:.1f}%")
    print(f"Missing embeddings: {vector_stats[4]/total*100:.1f}%")

## Metadata Analysis

In [ ]:
# Analyze metadata usage
cur.execute("""
    SELECT 
        jsonb_object_keys(metadata) as metadata_key,
        COUNT(*) as usage_count
    FROM document_chunks
    WHERE metadata != '{}'
    GROUP BY metadata_key
    ORDER BY usage_count DESC
    LIMIT 20
""")

metadata_keys = pd.DataFrame(
    cur.fetchall(),
    columns=['metadata_key', 'usage_count']
)

if not metadata_keys.empty:
    print("🏷️  METADATA KEY USAGE")
    print(metadata_keys.to_string(index=False))
    
    # Visualize metadata keys
    plt.figure(figsize=(10, 6))
    plt.barh(metadata_keys['metadata_key'][:10], metadata_keys['usage_count'][:10])
    plt.xlabel('Usage Count')
    plt.title('Top 10 Metadata Keys')
    plt.tight_layout()
    plt.show()

## Performance Insights

In [ ]:
# Test query performance
import time

def test_query_performance(query_type, query, params=None):
    """Test and time a query"""
    start_time = time.time()
    cur.execute(query, params)
    results = cur.fetchall()
    end_time = time.time()
    
    return {
        'query_type': query_type,
        'execution_time': end_time - start_time,
        'row_count': len(results)
    }

# Test various query types
performance_tests = []

# Simple count
performance_tests.append(test_query_performance(
    "Simple Count",
    "SELECT COUNT(*) FROM document_chunks"
))

# Filtered search
performance_tests.append(test_query_performance(
    "Filtered Search",
    "SELECT * FROM document_chunks WHERE topic = %s LIMIT 100",
    ('database',)
))

# Metadata search
performance_tests.append(test_query_performance(
    "Metadata Search",
    "SELECT * FROM document_chunks WHERE metadata @> %s LIMIT 100",
    (json.dumps({'file_type': 'pdf'}),)
))

# Vector similarity (if you have a sample vector)
try:
    cur.execute("SELECT dense_embedding FROM document_chunks WHERE dense_embedding IS NOT NULL LIMIT 1")
    sample_vector = cur.fetchone()
    if sample_vector:
        performance_tests.append(test_query_performance(
            "Vector Similarity",
            "SELECT * FROM document_chunks WHERE dense_embedding IS NOT NULL ORDER BY dense_embedding <=> %s LIMIT 10",
            (sample_vector[0],)
        ))
except:
    pass

# Display results
perf_df = pd.DataFrame(performance_tests)
print("⚡ QUERY PERFORMANCE TEST")
print("=" * 50)
for _, row in perf_df.iterrows():
    print(f"{row['query_type']}: {row['execution_time']*1000:.2f}ms ({row['row_count']} rows)")

## Storage Analysis

In [ ]:
# Get detailed storage information
cur.execute("""
    SELECT 
        'document_chunks' as table_name,
        pg_size_pretty(pg_table_size('document_chunks')) as table_size,
        pg_size_pretty(pg_indexes_size('document_chunks')) as indexes_size,
        pg_size_pretty(pg_total_relation_size('document_chunks')) as total_size,
        (pg_indexes_size('document_chunks')::float / pg_total_relation_size('document_chunks')::float * 100)::int as index_percentage
""")

storage_info = cur.fetchone()

print("💾 STORAGE BREAKDOWN")
print("=" * 50)
print(f"Table size: {storage_info[1]}")
print(f"Indexes size: {storage_info[2]}")
print(f"Total size: {storage_info[3]}")
print(f"Indexes are {storage_info[4]}% of total size")

# Visualize storage
labels = ['Table Data', 'Indexes']
sizes = [100 - storage_info[4], storage_info[4]]
colors = ['#ff9999', '#66b3ff']

plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Storage Distribution')
plt.axis('equal')
plt.show()

## Export Summary Report

In [ ]:
# Generate summary report
report = {
    'generated_at': datetime.now().isoformat(),
    'database_overview': stats_dict,
    'projects': project_stats.to_dict('records'),
    'top_documents': doc_stats.head(10).to_dict('records'),
    'topics': topic_stats.to_dict('records'),
    'vector_coverage': {
        'dense_coverage_pct': (vector_stats[1]/vector_stats[0]*100) if vector_stats[0] > 0 else 0,
        'sparse_coverage_pct': (vector_stats[2]/vector_stats[0]*100) if vector_stats[0] > 0 else 0,
        'both_coverage_pct': (vector_stats[3]/vector_stats[0]*100) if vector_stats[0] > 0 else 0
    },
    'performance_metrics': perf_df.to_dict('records')
}

# Save report
report_filename = f"pgvector_stats_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_filename, 'w') as f:
    json.dump(report, f, indent=2, default=str)

print(f"\n📄 Report saved to: {report_filename}")

## Cleanup

In [ ]:
# Close connections
cur.close()
conn.close()
rag.close()

print("Database connections closed")